In [56]:
import PyPDF2
from sentence_transformers import SentenceTransformer
import os

In [67]:
folder_path = r"C:\Users\Sudarshan\Desktop\files for project"
pdfpages = []
chunk_size = 200
chunk_overlap = 50 
text_chunks = []
for file in os.listdir(folder_path):
    file_path = os.path.join (folder_path , file)
    with open(file_path, 'rb') as f :
        reader = PyPDF2.PdfReader(f)
        num_pages = len(reader.pages)
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            text= page.extract_text()
            cleaned_text = text.replace('\n', ' ').strip()
            page_data = {
                "document_name":file,
                "page_number":page_number+1,
                "text": cleaned_text
            }
            pdfpages.append(page_data)

Chunking.....

In [68]:
for page_data in pdfpages:
    text = page_data["text"]
    words = text.split()

    #here we run chunking loop inside the page loop because we dont want the words of two files mix with eachother 
    for i in range (0 , len(words),chunk_size-chunk_overlap):
        chunk_words= words [i:i+chunk_size]
        chunk_text = " ".join(chunk_words)
        text_chunks.append({
            "document_name": page_data["document_name"],
            "page_number": page_data["page_number"],
            "chunk_index":(i//(chunk_size-chunk_overlap))+1,
            "text":chunk_text  
        })

Generating embedding

In [69]:
batch_size = 20
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') #384 dimensional encoding per chunk 
texts = [chunk["text"] for chunk in text_chunks ]
embeddings = model.encode(
    texts , 
    batch_size=20,
    show_progress_bar=True,
    normalize_embeddings=True
)

Batches: 100%|██████████| 14/14 [00:07<00:00,  1.86it/s]


Faiss Indexing and Semantic Search

In [70]:
import faiss
import numpy as np
import time

In [71]:
embeddings_array= np.array(embeddings, dtype='float32')
embedding_dim = embeddings_array.shape[1]
print(embeddings_array.shape)
#(271, 384)



(271, 384)


In [125]:
query = model.encode('what is ai ?')
query_embedding = query.reshape(1,-1)
k=6

In [ ]:
d=384
index= faiss.IndexFlatIP(d)
index.is_trained
index.add(embeddings_array) 
distances , indices= index.search(query_embedding, k)
print(indices[0])#[248 246  46  21  56 267]

True

In [ ]:
top_k_results = [
    f"Rank {rank+1}: Doc='{text_chunks[idx]['document_name']}', "
    f"Page={text_chunks[idx]['page_number']}, "
    f"Chunk={text_chunks[idx]['chunk_index']}\n"
    f"Text Preview: {text_chunks[idx]['text'][:100]}..."  # first 100 chars
    for rank, idx in enumerate(indices[0])
]

for i in top_k_results:
    print(i)

In [ ]:
%%time
nbits= 32
index = faiss.IndexLSH(d,nbits)
distances , indexes =index.search(query_embedding, k)
print(indexes[0])
#[-1 -1 -1 -1 -1 -1] (vector size is too small)


In [75]:
M= 32 
ef_search = 200 
ef_construction = 100 
index= faiss.IndexHNSWFlat(d,M)
index.hnsw.efSearch = ef_search
index.hnsw.efConstruction = ef_construction
index.add(embeddings_array)
distances , indexes =index.search(query_embedding, k)
print(indexes[0])
#[248 246  46  21  56 267]


[248 246  46  21  56 267]


In [ ]:
nlist= 5
quantizer= faiss.IndexFlatIP(384)
index= faiss.IndexIVFFlat(quantizer , 384 , nlist)
index.train(embeddings_array)
index.add(embeddings_array)
distances , indexes =index.search(query_embedding, k)
print(indexes[0])
#[248 246  46  56 267 249]





[248 246  46  56 267 249]
